<a href="https://colab.research.google.com/github/hargurjeet/Maersk/blob/main/Solution_Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q gitpython

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 5.1 MB/s eta 0:00:00


In [2]:
## Python Imports
import pandas as pd
import numpy as np
from git import Repo
import os

In [3]:
# Clone the GitHub repository
repo_url = 'https://github.com/hargurjeet/Maersk'
repo_dir = 'Maersk'
Repo.clone_from(repo_url, repo_dir)

<git.repo.base.Repo '/content/Maersk/.git'>

In [16]:
## Acessing the vessel data
file_path  = "/content/Maersk/Vessel_journey_data.csv"
date_cols = ['arrival_date', 'departure_date',
             'arrival_time', 'departure_time', 'prev_arrival_time', 'prev_departure_time', 'next_arrival_time']

vessel_data = pd.read_csv(file_path, parse_dates=date_cols)

vessel_data.head(2)

In [23]:
vessel_data.Port.head(6)

0             Xiamen
1        Los Angeles
2            Qingdao
3       Laem Chabang
4          Singapore
5    Tanjung Pelepas
Name: Port, dtype: object

In [12]:
vessel_data.columns

Index(['arrival_time_departure_time_prev_arrival_time_prev_departure_time_next_arrival_time',
       'call_id', 'arrival_date', 'departure_date', 'imo', 'vessel_name_ais',
       'vessel_type_ais', 'Country', 'CountryCode', 'Port', 'Terminal',
       'Duration', 'Service_name', 'length', 'vesteu', 'prev_port',
       'prev_duration', 'prev_leg_duration', 'prev_leg_distance_nm',
       'prev_leg_stationary_hours', 'prev_call_id', 'next_port',
       'next_duration', 'next_leg_duration', 'next_leg_distance_nm',
       'next_leg_stationary_hours', 'next_call_id'],
      dtype='object')

In [19]:
## acesssing geo data
geo_data = pd.read_csv("/content/Maersk/Geographical Data.csv")
geo_data.head()

,CountryCode,Country,PORT_UNLOCODE,PORT,LATITUDE,LONGITUDE
0,CZ,Czech Republic,CZKR2,Kropacova Vrutice,50.33060,14.71620
1,GB,United Kingdom,GB2WW,Farlington Hants,50.85000,-1.02000
2,RO,Romania,ROHN5,Sangeru,45.13238,26.34561
3,ES,Spain,ESFTC,Fontcoberta Girona,42.13000,2.80000
4,KG,Kyrgyzstan,KG6IA,Dzhalal-Abad,40.93400,73.00700


In [20]:
print(vessel_data.shape, geo_data.shape)

(1131992, 31) (142765, 6)


In [18]:
vessel_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1131992 entries, 0 to 1131991
Data columns (total 31 columns):
 #   Column                     Non-Null Count    Dtype              
---  ------                     --------------    -----              
 0   call_id                    1131992 non-null  object             
 1   arrival_date               1131992 non-null  datetime64[ns]     
 2   departure_date             1131992 non-null  datetime64[ns]     
 3   arrival_time               1131992 non-null  datetime64[ns, UTC]
 4   departure_time             1131992 non-null  datetime64[ns, UTC]
 5   imo                        1131992 non-null  int64              
 6   vessel_name_ais            1131992 non-null  object             
 7   vessel_type_ais            1131366 non-null  object             
 8   Country                    1131992 non-null  object             
 9   CountryCode                1130921 non-null  object             
 10  Port                       1131992 non-nul

In [21]:
geo_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 142765 entries, 0 to 142764
Data columns (total 6 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   CountryCode    142643 non-null  object 
 1   Country        142765 non-null  object 
 2   PORT_UNLOCODE  142765 non-null  object 
 3   PORT           142764 non-null  object 
 4   LATITUDE       142765 non-null  float64
 5   LONGITUDE      142765 non-null  float64
dtypes: float64(2), object(4)
memory usage: 6.5+ MB


In [24]:
## Combining both the datasets
vessel_geo_data = pd.merge(vessel_data, geo_data[['PORT', 'LATITUDE', 'LONGITUDE']], left_on='Port', right_on='PORT', how='left')
vessel_geo_data.head()

,call_id,arrival_date,departure_date,arrival_time,departure_time,imo,vessel_name_ais,vessel_type_ais,Country,CountryCode,...,next_port,next_arrival_time,next_duration,next_leg_duration,next_leg_distance_nm,next_leg_stationary_hours,next_call_id,PORT,LATITUDE,LONGITUDE
0,7b75a18f54d78c79df4e9ceb29c65c60d6b5c7ae,2022-01-05,2022-01-07,2022-01-05 23:00:19+00:00,2022-01-07 21:04:26+00:00,9525376,SAFMARINE CHAMBAL,Container Ship,China,CN,...,Los Angeles,2022-02-10 12:01:14+00:00,207.020556,806.946667,7407.903024,201.0,c77797c9b9822d7a799e6f7664f7d6be5f73f270,Xiamen,24.450000,118.080000
1,c77797c9b9822d7a799e6f7664f7d6be5f73f270,2022-02-10,2022-02-19,2022-02-10 12:01:14+00:00,2022-02-19 03:02:28+00:00,9525376,SAFMARINE CHAMBAL,Container Ship,United States,US,...,Qingdao,2022-03-08 22:27:41+00:00,105.949167,427.420278,5967.080241,25.0,0dccb61f51069f40de6592464459777a2eb73334,Los Angeles,-37.462000,-72.356000
2,c77797c9b9822d7a799e6f7664f7d6be5f73f270,2022-02-10,2022-02-19,2022-02-10 12:01:14+00:00,2022-02-19 03:02:28+00:00,9525376,SAFMARINE CHAMBAL,Container Ship,United States,US,...,Qingdao,2022-03-08 22:27:41+00:00,105.949167,427.420278,5967.080241,25.0,0dccb61f51069f40de6592464459777a2eb73334,Los Angeles,34.052234,-118.243685
3,0dccb61f51069f40de6592464459777a2eb73334,2022-03-08,2022-03-13,2022-03-08 22:27:41+00:00,2022-03-13 08:24:38+00:00,9525376,SAFMARINE CHAMBAL,Container Ship,China,CN,...,Laem Chabang,2022-06-18 04:02:02+00:00,32.013611,2323.623333,3619.205422,1541.0,78668c935841770938abab12d821208372373e3f,Qingdao,36.083340,120.350000
4,78668c935841770938abab12d821208372373e3f,2022-06-18,2022-06-19,2022-06-18 04:02:02+00:00,2022-06-19 12:02:51+00:00,9525376,MAERSK CHAMBAL,Container Ship,Thailand,TH,...,Singapore,2022-06-23 03:00:26+00:00,28.002500,86.959722,805.756665,4.0,704ba180425814231bfd45db00d1ad3dd15e7791,Laem Chabang,13.382173,100.991433
